### Initiate llama-index OCI client

In [1]:
from llama_index.llms.oci_genai import OCIGenAI

llm = OCIGenAI(
    model="meta.llama-3.1-70b-instruct", # Switch to different models for testing
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.tenancy.oc1..aaaaaaaaumuuscymm6yb3wsbaicfx3mjhesghplvrvamvbypyehh5pgaasna",
    auth_type="SECURITY_TOKEN",
    auth_profile="BoatOc1", # Please update here with your own profile name
    additional_kwargs={"max_tokens": 1000}
)

### Simple Chat

In [2]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="Tell me a story"),
]
response = llm.chat(messages)
print(response)

assistant: Arrrr, settle yerself down with a pint o' grog and listen close, me hearty! I've got a tale for ye that'll make yer timbers shiver.

It were a dark and stormy night, and me and me crew, the "Maverick's Revenge," were sailin' through treacherous waters. The winds were howlin' like a pack o' wolves, and the lightnin' were flashin' like a thousand torches in the sky. Me and me mate, Barnaby Blackheart, were at the helm, tryin' to keep the ship on course.

Suddenly, a mighty wave crashed over the bow, and we were swept off our feet. I found meself clingin' to the riggin' for dear life, me boots fillin' up with seawater. Barnaby were clingin' to me leg, his eyes wide with fear.

"Cap'n, we're goin' down!" he shouted above the din o' the storm.

But I weren't havin' it. I've been sailin' these seas for nigh on 20 years, and I knew me ship like the back o' me hand. I gave a mighty roar and shouted, "Not on me watch, matey!"

I grabbed hold o' the wheel and started to turn it, fight

In [4]:
print(response.raw)

{'status': 200, 'headers': {'content-type': 'application/json', 'opc-request-id': 'C9E9A28AD1F1481C9D8332D47ED271F5/4287D9DCD289D6FDB46A9095BC7210FA/B45847257D96B1D26B11E802A2AB6F53', 'content-encoding': 'gzip', 'content-length': '1730'}, 'data': {
  "chat_response": {
    "api_format": "GENERIC",
    "choices": [
      {
        "finish_reason": "stop",
        "index": 0,
        "logprobs": {
          "text_offset": null,
          "token_logprobs": null,
          "tokens": null,
          "top_logprobs": null
        },
        "message": {
          "content": [
            {
              "text": "Arrrr, settle yerself down with a pint o' grog and listen close, me hearty! I've got a tale for ye that'll make yer timbers shiver.\n\nIt were a dark and stormy night, and me and me crew, the \"Maverick's Revenge,\" were sailin' through treacherous waters. The winds were howlin' like a pack o' wolves, and the lightnin' were flashin' like a thousand tiny lanterns in the sky. Me and me 

### Chat Streaming

In [3]:
resp = llm.stream_chat(messages)
for r in resp:
    print(r.delta, end="")

Arrrr, settle yerself down with a pint o' grog and listen close, me hearty! I've got a tale for ye that'll make yer timbers shiver.

It were a dark and stormy night, and me and me crew, the "Maverick's Revenge," were sailin' through treacherous waters. We'd been at sea for weeks, searchin' for the treasure of the infamous Captain Blackbeak. Legend had it that Blackbeak had hidden his loot on a remote island, guarded by sea monsters and cursed spirits.

As we navigated through the choppy waters, our lookout, Barnaby Blackheart, spied somethin' on the horizon. "Land ho!" he cried, pointin' to a tiny island risin' up out o' the sea like a giant's fist.

We dropped anchor and set off in the longboats, ready to face whatever dangers lay ahead. The island were a right proper jungle, with vines as thick as me arm and trees that seemed to stretch up to the clouds. We hacked our way through the underbrush, our cutlasses slicin' through the air like hot knives through butter.

As we reached the 

### Chat Chaining (Chat Prompt with a Template)

In [4]:
# Set up a chat prompt with a template
prompt_text = "Tell me a joke about {topic}"
formatted_prompt = prompt_text.format(topic="dogs")

# Send the prompt to OCIGenAI
response = llm.chat([ChatMessage(role="user", content=formatted_prompt)])
print(response.message.content)

Here's one:

Why did the dog go to the vet?

Because he was feeling ruff!

(Sorry, I know it's a paws-itively terrible pun, but I hope it made you howl with laughter!)


### Chat with Documents 

* Note: requires embed model set up for llama index

In [14]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.prompts import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

# Define documents as instances of the Document class
documents = [
    Document(text="Oracle database services provide high-performance versions of Oracle Database, including Oracle Autonomous Database to simplify relational database environments."),
    Document(text="Oracle Cloud Infrastructure (OCI) offers secure, scalable cloud services across compute, storage, and networking, supporting enterprise applications."),
    Document(text="Oracle Autonomous Database is a self-driving, self-securing, and self-repairing database that automates database maintenance tasks.")
]

# Create an index directly from the list of Document instances
index = VectorStoreIndex.from_documents(documents)

# Define a custom prompt template for condensing follow-up questions
custom_prompt = PromptTemplate(
    """
    Given a conversation (between Human and Assistant) and a follow-up message from Human,
    rewrite the message to be a standalone question that captures all relevant context
    from the conversation.
    <Chat History>
    {chat_history}
    <Follow-Up Message>
    {question}
    <Standalone Question>
    """
)

# Initialize chat history with prior messages
chat_history = [
    ChatMessage(role=MessageRole.SYSTEM, content="You are an AI assistant."),
    ChatMessage(role=MessageRole.USER, content="Tell me something about Oracle."),
    ChatMessage(role=MessageRole.ASSISTANT, content="Oracle is a leading provider of database solutions."),
]

# Create a query engine for document retrieval
query_engine = index.as_query_engine()

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=chat_history,
    verbose=True,
)

follow_up_question = "What are its main products?"

response = chat_engine.chat(follow_up_question)
print(response)

### Basic tool calling in llamaindex 

only Cohere supports tool calling for now

In [5]:
from llama_index.llms.oci_genai import OCIGenAI
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.tools import FunctionTool

llm = OCIGenAI(
    model="cohere.command-r-plus",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.tenancy.oc1..aaaaaaaaumuuscymm6yb3wsbaicfx3mjhesghplvrvamvbypyehh5pgaasna",  
    auth_type="SECURITY_TOKEN",
    auth_profile="BoatOc1",
)

In [6]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Addition function on two integers."""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

response = llm.chat_with_tools(
    tools=[add_tool, multiply_tool],
    user_msg="What is 3 * 12? Also, what is 11 + 49?",
)

print(response)

assistant: I will use the multiply and add tools to answer the question.


In [7]:
print(dict(response))

{'message': ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='I will use the multiply and add tools to answer the question.', additional_kwargs={'finish_reason': 'COMPLETE', 'documents': None, 'citations': None, 'search_queries': None, 'is_search_required': None, 'tool_calls': [{'toolUseId': 'a387ebd8eb1346c296e0732d611ff981', 'name': 'multiply', 'input': '{"a": 3, "b": 12}'}, {'toolUseId': 'a0d0589161ae4b02a7b3028f4c23c60c', 'name': 'add', 'input': '{"a": 11, "b": 49}'}]}), 'raw': {'status': 200, 'headers': {'content-type': 'application/json', 'opc-request-id': '4C8C0553B2B44ED18FE7CF48021654F5/088D8729A525795E77C1E6A2A3D9AEB3/CBBD3C8D8C90E13D6EED0E1A9144B83C', 'content-encoding': 'gzip', 'content-length': '304'}, 'data': {
  "chat_response": {
    "api_format": "COHERE",
    "chat_history": [
      {
        "message": "What is 3 * 12? Also, what is 11 + 49?",
        "role": "USER"
      },
      {
        "message": "I will use the multiply and add tools to answer the

In [8]:
tool_calls = response.message.additional_kwargs.get('tool_calls', [])
print(tool_calls)

[{'toolUseId': 'a387ebd8eb1346c296e0732d611ff981', 'name': 'multiply', 'input': '{"a": 3, "b": 12}'}, {'toolUseId': 'a0d0589161ae4b02a7b3028f4c23c60c', 'name': 'add', 'input': '{"a": 11, "b": 49}'}]


### Chat streaming with tools

In [9]:
# Use streaming with tools
gen = llm.stream_chat_with_tools(
    tools=[multiply_tool, add_tool],
    user_msg="What is 23 * 45?",
)

# Process the stream
try:
    for response in gen:
        # Print content as it arrives
        if response.message.content:
            print(f"Content: {response.message.content}")

        # Check for tool calls
        if "tool_calls" in response.message.additional_kwargs:
            tool_calls = response.message.additional_kwargs["tool_calls"]
            print(f"Tool Calls: {tool_calls}")
except Exception as e:
    print(f"Error in stream processing: {e}")

Content: I
Content: I will
Content: I will use
Content: I will use the
Content: I will use the multiplication
Content: I will use the multiplication tool
Content: I will use the multiplication tool to
Content: I will use the multiplication tool to find
Content: I will use the multiplication tool to find the
Content: I will use the multiplication tool to find the answer
Content: I will use the multiplication tool to find the answer.
Content: I will use the multiplication tool to find the answer.I will use the multiplication tool to find the answer.
Tool Calls: [{'toolUseId': 'a2ef38fed43a4813979b0d3f136e5d86', 'name': 'multiply', 'input': '{"a": 23, "b": 45}'}]
Content: I will use the multiplication tool to find the answer.I will use the multiplication tool to find the answer.
Tool Calls: [{'toolUseId': '7e51be5c36d245389f54b50b21515dc8', 'name': 'multiply', 'input': '{"a": 23, "b": 45}'}]


### Invoking tools and passing tool outputs back to model

In [10]:
# Initial user query
query = "What is 3 * 12? Also, what is 11 + 49?"
user_message = ChatMessage(role=MessageRole.USER, content=query)

# Pass initial message to the LLM with tool calling enabled
response = llm.chat_with_tools(tools=[add_tool, multiply_tool], user_msg=user_message.content)
print(response.message.content)  # Initial assistant response about using tools

# Extract tool calls and invoke each tool based on the response
messages = [user_message, response.message]  # Start with user message and initial assistant response

for tool_call in response.message.additional_kwargs.get("tool_calls", []):
    # Select the tool based on the tool call name
    selected_tool = {"add": add_tool, "multiply": multiply_tool}[tool_call["name"].lower()]
    tool_input = eval(tool_call["input"])  # Convert input JSON string to dict
    tool_output = selected_tool(**tool_input)  # Invoke tool

    # Add the tool's output as a new message in the conversation
    tool_output_message = ChatMessage(
        role=MessageRole.TOOL,
        content=str(tool_output),  # Format tool output as string
        additional_kwargs={"tool_call_id": tool_call.get("toolUseId")}
    )
    messages.append(tool_output_message)

# Pass the messages back to the LLM for a final response using the tool outputs
final_response = llm.chat(messages)
print(final_response)

I will use the multiply tool to calculate 3 * 12 and the add tool to calculate 11 + 49.
assistant: 3 * 12 is **36** and 11 + 49 is **60**.


### Few-shot tool prompting & basic agent-like tool looping

In [11]:
import json
# Define initial few-shot messages
messages = [
    ChatMessage(
        role=MessageRole.USER,
        content="What's the product of 317253 and 128472 plus four"
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="",
        additional_kwargs={
            "tool_calls": [{"name": "multiply", "input": {"x": 317253, "y": 128472}, "toolUseId": "1"}]
        }
    ),
    ChatMessage(
        role=MessageRole.TOOL,
        content="16505054784",
        additional_kwargs={"tool_call_id": "1"}
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="",
        additional_kwargs={
            "tool_calls": [{"name": "add", "input": {"x": 16505054784, "y": 4}, "toolUseId": "2"}]
        }
    ),
    ChatMessage(
        role=MessageRole.TOOL,
        content="16505054788",
        additional_kwargs={"tool_call_id": "2"}
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="The product of 317253 and 128472 plus four is 16505054788"
    ),
]

# Define the query
query = "Whats 119 times 7 plus 30. Don't do any math yourself, only use tools for math. Respect order of operations. when you have the answer output 'the answer is: <answer>'"
messages.append(ChatMessage(role=MessageRole.USER, content=query))

# Start agent-like tool looping
while True:
    ai_response = llm.chat_with_tools(messages=messages, tools=[multiply_tool, add_tool])
    print(ai_response.message.content)

    if "the answer is:" in ai_response.message.content.lower():
        break

    messages.append(ai_response.message)

    # Execute tools based on tool calls and append tool responses
    if ai_response.message.additional_kwargs.get("tool_calls"):
        for tool_call in ai_response.message.additional_kwargs["tool_calls"]:
            tool_name = tool_call["name"]
            tool_input = tool_call["input"]

            # If tool_input is a string, parse it as JSON
            if isinstance(tool_input, str):
                tool_input = json.loads(tool_input)

            # Select tool and invoke it
            if tool_name == "multiply":
                result = multiply(**tool_input)
            elif tool_name == "add":
                result = add(**tool_input)

            # Append the tool result as a tool message
            messages.append(
                ChatMessage(
                    role=MessageRole.TOOL,
                    content=str(result),
                    additional_kwargs={"tool_call_id": tool_call["toolUseId"]}
                )
            )


I will use the multiply tool to multiply 119 and 7, then use the add tool to add 30 to the result.
119 x 7 = 833. Now I will add 30 to this.
The answer is: 863


### Create a llama-index agent

In [12]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings

# Set the LLM in the global settings
Settings.llm = llm

# Define your tools
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the result."""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and return the result."""
    return a + b

# Create FunctionTool instances
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
tools = [multiply_tool, add_tool]

# Create the ReActAgent
agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
)

# Start the conversation
response = agent.chat(
    "Whats 119 times 7 plus 30. Don't do any math yourself, only use tools for math. Respect order of operations."
)
print(response)


> Running step 69ad5aaa-c63d-4d2d-a06d-ed4df7907ebb. Step input: Whats 119 times 7 plus 30. Don't do any math yourself, only use tools for math. Respect order of operations.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 119, 'b': 7}
Observation: 833
> Running step 58e39756-5bba-4bf5-9734-02cc9b342288. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The answer is **863**.
The answer is **863**.
